In [27]:
import time
import os
import re
import socket
import datetime
import pyautogui

def return_datetime():
    """
    Provides a string of the date and time in the format YYYY/MM/DD HH:MM:SS
    Requires:
    Nothing
    
    Returns:
    date_time (string): Returns the date and time in the following format YYYY/MM/DD HH:MM:SS
    """
    
    date_time = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S')
    return date_time

def packet(obj):
    """
    Converts an obj to bytes in order to send them via TCP/IP, requires obj (any type compatible with str() function)
    Requires:
    * obj (any type except None): Takes any type of object that can be converted to string
    
    Returns:
    * obj (bytes): Returns the input object, converted to bytes
    """
    
    obj = str(obj)
    return bytes(obj, 'utf-8')

def change_attenuator(value):
    """
    Attempts to change the attenuator value by moving the mouse to the Laser tab in the program, then entering the 
    attenuator value and finally clicking in set position
    Requires:
    * value (float): The new attenuator value
    
    Returns:
    Nothing
    """
    
    try:
        #take a screenshot
        im=pyautogui.screenshot()
        #Locate the laser tab
        pyautogui.moveTo(200,200)
        obj = pyautogui.locateOnScreen('laser_tab.bmp') 
        pyautogui.moveTo(obj[0], obj[1], 0.05) #Moves the mouse to the upper left corner of the obj
        pyautogui.moveRel(obj[2]/2, obj[3]/2, 0.05) #Moves the mouse to the middle of the detected obj
        pos = pyautogui.position()
        pyautogui.click(pos[0], pos[1], button = 'left') #Clicks on the tab

        #In laser tab find the att position tab
        pyautogui.moveTo(200,200)
        obj = pyautogui.locateOnScreen('degrees_sp.bmp')
        pyautogui.moveTo(obj[0], obj[1], 0.05) #Moves the mouse to the upper left corner of the obj
        pyautogui.moveRel(10, obj[3] * 3/4, 0.05) #Moves the mouse to the middle in y direction and 10 pixels right
        pos = pyautogui.position()
        pyautogui.click(pos[0], pos[1], button = 'left') #Clicks on the field to modify it
        pyautogui.typewrite(['delete', 'delete','delete', 'delete']) #Deletes current field
        pyautogui.typewrite(value) #Types new attenuator value

        #Finally click the set position
        pyautogui.moveTo(200,200)
        obj = pyautogui.locateOnScreen('set_position.bmp')
        pyautogui.moveTo(obj[0], obj[1], 0.05) #Moves the mouse to the upper left corner of the obj
        pyautogui.moveRel(obj[2]/2, obj[3]/2, 0.05) #Moves the mouse to the middle of the detected obj
        pos = pyautogui.position()
        pyautogui.click(pos[0], pos[1], button = 'left') #Clicks on the field to modify it
        return
    except:
        print("except clause entered in change_attenuator(): program closed or minimized", end = '\r')
        return


def create_server(ip_addr, port, BUFFER_SIZE = 32):
    """
    Function to create a server that awaits connections from the client computer
    Requires:
    ip_addr (string): A string with the ip address of the server
    port (integer): The port number on which to receive the connections
    BUFFER_SIZE (integer): The size in bytes of each packet sent
    
    Returns:
    Nothing
    """
    
    try:
        TCP_IP = ip_addr
        TCP_PORT = port
        prev_att_val = ''
        #BUFFER_SIZE establishes maximum length per packet
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.bind((TCP_IP, TCP_PORT))
        s.listen(1) #Number of connections that can be queued in current port, max size is 5
        conn, addr = s.accept()
        print("Connection established, address: ", addr)
        while True:
            data = conn.recv(BUFFER_SIZE)
            if not data:
                print("\nConnection ended by client")
                break;
            str_data = str(data)
            if str_data != prev_att_val: 
                try:
                    string = "{}, data received = {}".format(return_datetime(), str_data)
                    string2 = "changed attenuator to {}    ".format(str_data)
                    print(string+", "+string2, end='\r')
                    change_attenuator(str_data) #Moves mouse and keyboard to change the attenuator value
                    
                except:
                    string = "\n{}, data received = {}".format(return_datetime(), str_data)
                    string2 = "failed to change attenuator. Program closed or minimized."
                    print(string+", "+string2, end='\r')
                    continue

            prev_att_val = str_data
            conn.send(packet(return_datetime()))
        conn.close()
        s.close()
    except:
        print("Except clause entered in create_server(): attempting to close connections...")
        conn.close()
        s.close()
    return


In [28]:
create_server('127.0.0.1', 13539)   


Connection established, address:  ('127.0.0.1', 60334)
Except clause entered in create_server(): attempting to close connections... 
